In [14]:
from modules.utils import *
from modules.prompt import *
from langchain_teddynote.models import MultiModal
from langchain_core.output_parsers import StrOutputParser


### 파일 분리

In [2]:
filepath = ".cache\\files\\seq2seq.pdf"
file_paths,basename = split_pdf(filepath)

print(file_paths,basename)

분할 PDF 생성: .cache\files\seq2seq\seq2seq_0000_0004.pdf
분할 PDF 생성: .cache\files\seq2seq\seq2seq_0005_0008.pdf
['.cache\\files\\seq2seq\\seq2seq_0000_0004.pdf', '.cache\\files\\seq2seq\\seq2seq_0005_0008.pdf'] .cache\files\seq2seq


In [3]:
analysis_file_path = upstage_layout_analysis(file_paths)

In [4]:
analysis_file_path

['.cache\\files\\seq2seq\\seq2seq_0000_0004.json',
 '.cache\\files\\seq2seq\\seq2seq_0005_0008.json']

In [5]:
element_content = reOrder_id(analysis_file_path)

In [6]:
_element_content = extract_image(element_content,basename)

base64_encoding params 없습니다.
이미지 태그가 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
Saved image: .cache\files\seq2seq\image_12.png
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_enco

In [7]:
image_urls,system_prompts,user_prompts = order_image_prompt(_element_content)

In [8]:
llm = get_gpt()
multimodal_llm = MultiModal(llm)
answer = multimodal_llm.batch(
    image_urls, system_prompts, user_prompts, display_image=False
)

In [9]:
change_element_content = change_image_text_to_summary(answer,_element_content)

id 값: 12
id 값: 68
id 값: 69
id 값: 75


### 방정식 설명 추가 로직

In [10]:
system_prompts,user_prompts = order_equation_prompt(change_element_content)

In [11]:


answer = request_llm_equation(system_prompts, user_prompts)

In [12]:
final_content = add_eqaution_description(answer,change_element_content)

id 값: 22
id 값: 26
id 값: 38
id 값: 40


In [16]:
user_input = "이논문이 해결하고자하는 문제가머야 ?"
docs = []
for d in final_content:
    doc = Document(page_content=d["content_text"], metadata=d["metadata"])
    docs.append(doc)

faiss_retriever = get_retriever(docs)
bm25_retriever = get_bm25_retriever(docs)
esenmble_retriever = get_esenmble_retriever(faiss_retriever, bm25_retriever)
rerank = get_reranker(esenmble_retriever, user_input)
reorder_context = reorder_documents(rerank)
prompt = get_prompt_user_request()
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": user_input, "context": reorder_context})
print(response)

# 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스** $(x_1, \cdots, x_T)$에 대해 **출력 시퀀스** $Y_1, \cdots, Y_{T'}$의 조건부 확률 $p(Y_1, \cdots, Y_{T'} | x_1, \cdots, x_T)$를 추정하는 문제를 다룹니다. 여기서 입력 시퀀스와 출력 시퀀스의 길이 $T$와 $T'$는 서로 다를 수 있습니다. (출처: 3페이지)

---

## 문제의 핵심

- **입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있음**  
  예를 들어, 영어 문장(입력)이 있고, 그에 대응하는 프랑스어 문장(출력)이 있는데, 두 문장의 단어 수가 다를 수 있습니다.

- **조건부 확률 추정**  
  LSTM은 입력 시퀀스가 주어졌을 때, 출력 시퀀스가 나올 확률을 계산합니다. 이 확률은 다음과 같이 표현됩니다:

  $$
  p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T) = \prod_{t=1}^{T'} p(y_t | v, y_1, \cdots, y_{t-1})
  $$

  여기서 $v$는 입력 시퀀스 $(x_1, \cdots, x_T)$를 LSTM의 마지막 은닉 상태로 요약한 고정 차원 벡터입니다. 즉, LSTM은 입력 전체를 하나의 벡터로 압축한 뒤, 그 벡터와 지금까지 생성한 출력 단어들을 바탕으로 다음 단어를 예측합니다. (출처: 3페이지, 식 26)

---

## 쉽게 이해하기 위한 비유

- **그림책 이야기하기**  
  그림책에 여러 장의 그림이 있다고 생각해보세요. 이 그림들이 입력 시퀀스 $(x_1, \cdots, x_T)$입니다.  
  LSTM은 이 그림들을 모두 보고, 그 내용을 하나의 요약본 $v$로 만듭니다.  
  그리고 이 요약본을 보면서 한 단어씩 이야기를 만들어 나갑니다.  
  첫 번째 단어를 말할 때는 요약본만 보고, 두 번째 단어를 말할 때는 요약본과 첫 번째 단어

# 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스** $(x_1, \cdots, x_T)$에 대해 **출력 시퀀스** $Y_1, \cdots, Y_{T'}$의 조건부 확률 $p(Y_1, \cdots, Y_{T'} | x_1, \cdots, x_T)$를 추정하는 문제를 다룹니다. 여기서 입력 시퀀스와 출력 시퀀스의 길이 $T$와 $T'$는 서로 다를 수 있습니다. (출처: 3페이지)

---

## 문제의 핵심

- **입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있음**  
  예를 들어, 영어 문장(입력)이 있고, 그에 대응하는 프랑스어 문장(출력)이 있는데, 두 문장의 단어 수가 다를 수 있습니다.

- **조건부 확률 추정**  
  LSTM은 입력 시퀀스가 주어졌을 때, 출력 시퀀스가 나올 확률을 계산합니다. 이 확률은 다음과 같이 표현됩니다:

  $$
  p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T) = \prod_{t=1}^{T'} p(y_t | v, y_1, \cdots, y_{t-1})
  $$

  여기서 $v$는 입력 시퀀스 $(x_1, \cdots, x_T)$를 LSTM의 마지막 은닉 상태로 요약한 고정 차원 벡터입니다. 즉, LSTM은 입력 전체를 하나의 벡터로 압축한 뒤, 그 벡터와 지금까지 생성한 출력 단어들을 바탕으로 다음 단어를 예측합니다. (출처: 3페이지, 식 26)

---

## 쉽게 이해하기 위한 비유

- **그림책 이야기하기**  
  그림책에 여러 장의 그림이 있다고 생각해보세요. 이 그림들이 입력 시퀀스 $(x_1, \cdots, x_T)$입니다.  
  LSTM은 이 그림들을 모두 보고, 그 내용을 하나의 요약본 $v$로 만듭니다.  
  그리고 이 요약본을 보면서 한 단어씩 이야기를 만들어 나갑니다.  
  첫 번째 단어를 말할 때는 요약본만 보고, 두 번째 단어를 말할 때는 요약본과 첫 번째 단어를 기억해서 말하는 식입니다.  
  이렇게 한 단어씩 차례대로 말하면서 전체 이야기를 완성하는 것이죠.

- **색깔 블록 이름 맞추기**  
  여러 색깔 블록이 줄지어 있고, 이 블록들의 색깔을 하나씩 말해야 한다고 상상해보세요.  
  먼저 블록 전체를 보고 색깔들의 요약본 $v$를 만듭니다.  
  그리고 첫 번째 색깔을 말하고, 두 번째 색깔을 말할 때는 요약본과 첫 번째 색깔을 기억해서 말합니다.  
  이런 식으로 모든 색깔을 맞추는 확률을 계산하는 것이 이 문제입니다.

---

## 문제 해결의 중요성

- **길이가 다른 시퀀스 간의 매핑 문제**  
  입력과 출력 시퀀스의 길이가 다를 때, 어떻게 정확하게 대응시키고 예측할지 어려움이 있습니다.

- **장기 의존성 문제**  
  LSTM은 긴 시퀀스에서도 중요한 정보를 잃지 않고 기억할 수 있어, 긴 문장이나 복잡한 시퀀스 문제에 적합합니다.

- **기계 번역, 음성 인식 등 다양한 분야에 적용 가능**  
  이 문제를 잘 해결하면 자연어 처리, 음성 인식, 동작 인식 등 여러 분야에서 성능 향상을 기대할 수 있습니다.

---

# 요약

| 항목 | 설명 |
|---|---|
| **해결하고자 하는 문제** | 입력 시퀀스 $(x_1, \cdots, x_T)$가 주어졌을 때, 길이가 다를 수 있는 출력 시퀀스 $Y_1, \cdots, Y_{T'}$의 조건부 확률을 정확히 추정하는 것 |
| **핵심 아이디어** | 입력 시퀀스를 LSTM의 마지막 은닉 상태로 요약한 벡터 $v$를 만들고, 이를 바탕으로 출력 시퀀스를 한 단계씩 생성하며 확률을 계산 |
| **중요성** | 시퀀스 길이가 다르고 복잡한 문제에 대해 효과적인 모델링 가능, 기계 번역 등 다양한 응용 분야에 활용 가능 |

---

필요하면 추가 설명이나 다른 부분도 알려주세요!